In [ ]:
def blocking_variance(series, min_blocks=4):
    """
    Return
        bin_sizes  – powers-of-2 block lengths
        var_mean   – variance of the MCMC mean at each block size
        err_var    – 1-σ uncertainty of that variance
    """
    N = len(series)
    max_power = int(np.log2(N // min_blocks))
    bin_sizes = 2 ** np.arange(max_power + 1)

    var_mean, err_var = [], []
    for L in bin_sizes:
        n_blocks = N // L
        if n_blocks < min_blocks:               # need ≥ min_blocks estimates
            break
        # reshape   → one mean per block
        blocks = series[:n_blocks * L].reshape(n_blocks, L).mean(axis=1)

        s2      = blocks.var(ddof=1)            # unbiased σ² of block means
        v_mean  = s2 / n_blocks                 # Var[overall mean]
        var_mean.append(v_mean)

        # 1-σ error bar from χ² statistics
        err_var.append(v_mean * np.sqrt(2 / (n_blocks - 1)))

    return bin_sizes[:len(var_mean)], np.array(var_mean), np.array(err_var)

In [ ]:
# --- 0. parameters ----------------------------------------

N = 100000  # Number of samples
step_size = 1  # Step size for the proposal distribution
xo = rng.uniform(-step_size, step_size)  # Initial point for the Markov chain rng.uniform(-step_size, step_size)
burnin = 1000  # Number of samples to discard as burn-in
thinning = 2  # Thinning factor for the Markov chain

# --- 1. generate a correlated chain (AR(1)) -----------------
markov_chain, prob_accept = metropolis_hastings(xo, thinning, burnin, N, step_size)

f_i = f(markov_chain)/ target_pdf(markov_chain)  # Importance weights for the samples

# --- 2. blocking analysis with error bars ------------------
bin_sizes, vars_mean, errs = blocking_variance(f_i)

# --- 3. plot -----------------------------------------------
plt.figure(figsize=(8, 6))
plt.errorbar(bin_sizes, vars_mean, yerr=errs,
             fmt='-s', capsize=3, linewidth=1, markersize=4)
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Bin size $L$")
plt.ylabel(r"Var[$\hat\mu$]")
plt.title("Variance of MCMC mean vs. block size\n(with 1-σ error bars)")
plt.tight_layout()
plt.show()
